# Configuration

In [9]:
import os
from os.path import join
import torch
from enum import Enum
# from parse import parse_args
import multiprocessing

# os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
# args = parse_args()

# ROOT_PATH = os.path.dirname(os.path.dirname(__file__))
# CODE_PATH = join(ROOT_PATH, 'code')
# DATA_PATH = join(ROOT_PATH, 'data')
# BOARD_PATH = join(CODE_PATH, 'runs')
# FILE_PATH = join(CODE_PATH, 'checkpoints')
# import sys
# sys.path.append(join(CODE_PATH, 'sources'))


# if not os.path.exists(FILE_PATH):
#     os.makedirs(FILE_PATH, exist_ok=True)

config = {}
all_dataset = ['lastfm', 'gowalla', 'yelp2018', 'amazon-book']
all_models  = ['mf', 'lgn']
# config['batch_size'] = 4096
config['bpr_batch_size'] = 2048 # the batch size for bpr loss training procedure"
config['latent_dim_rec'] = 64 # the embedding size of lightGCN
config['lightGCN_n_layers']= 3 # the layer num of lightGCN
config['dropout'] = 0 # using the dropout or not
config['keep_prob']  = 0.6 # the batch size for bpr loss training procedure
config['A_n_fold'] = 100 # the fold num used to split large adj matrix, like gowalla
config['test_u_batch_size'] = 100 # the batch size of users for testing
config['multicore'] = 0 # whether we use multiprocessing or not in test
config['lr'] = 0.001 # the learning rate
config['decay'] = 1e-4 # the weight decay for l2 normalizaton
config['pretrain'] = 0 # whether we use pretrained weight or not
config['A_split'] = False #
config['bigdata'] = False #

GPU = torch.cuda.is_available()
device = torch.device('cuda' if GPU else "cpu")
CORES = multiprocessing.cpu_count() // 2
seed = 2020

dataset = 'gowalla'
model_name = 'lgn'
if dataset not in all_dataset:
    raise NotImplementedError(f"Haven't supported {dataset} yet!, try {all_dataset}")
if model_name not in all_models:
    raise NotImplementedError(f"Haven't supported {model_name} yet!, try {all_models}")




TRAIN_epochs = 1000
LOAD = 0
PATH = "./checkpoints" # path to save weights
topks = eval("[20]") # @k test list
tensorboard = 1 # enable tensorboard
comment = "lgn"
# let pandas shut up
from warnings import simplefilter
simplefilter(action="ignore", category=FutureWarning)


# Data Loader

In [7]:
import os
from os.path import join
import sys
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix
import scipy.sparse as sp
from time import time

# abstracted class for every data
# class BasicDataset(Dataset):
#     def __init__(self):
#         print("init dataset")
    
#     # @property - getter / @variable.setter - setter
#     @property
#     def n_users(self):
#         raise NotImplementedError
    
#     @property
#     def m_items(self):
#         raise NotImplementedError
    
#     @property
#     def trainDataSize(self):
#         raise NotImplementedError
    
#     @property
#     def testDict(self):
#         raise NotImplementedError
    
#     @property
#     def allPos(self):
#         raise NotImplementedError
    
#     def getUserItemFeedback(self, users, items):
#         raise NotImplementedError
    
#     def getUserPosItems(self, users):
#         raise NotImplementedError
    
#     def getUserNegItems(self, users):
#         """
#         not necessary for large dataset
#         it's stupid to return all neg items in super large dataset
#         """
#         raise NotImplementedError
    
#     def getSparseGraph(self):
#         """
#         build a graph in torch.sparse.IntTensor.
#         Details in NGCF's matrix form
#         A = 
#             |I,   R|
#             |R^T, I|
#         """
#         raise NotImplementedError
    

In [ ]:
def _split_A_hat(self,A):
    A_fold = []
    fold_len = (self.n_users + self.m_items) // self.folds
    for i_fold in range(self.folds):
        start = i_fold*fold_len
        if i_fold == self.folds - 1:
            end = self.n_users + self.m_items
        else:
            end = (i_fold + 1) * fold_len
        A_fold.append(self._convert_sp_mat_to_sp_tensor(A[start:end]).coalesce().to(world.device))
    return A_fold

def _convert_sp_mat_to_sp_tensor(self, X):
    coo = X.tocoo().astype(np.float32)
    row = torch.Tensor(coo.row).long()
    col = torch.Tensor(coo.col).long()
    index = torch.stack([row, col])
    data = torch.FloatTensor(coo.data)
    return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))
    
def getSparseGraph(self):
    print("loading adjacency matrix")
    if self.Graph is None:
        try:
            pre_adj_mat = sp.load_npz(self.path + '/s_pre_adj_mat.npz')
            print("successfully loaded...")
            norm_adj = pre_adj_mat
        except :
            print("generating adjacency matrix")
            s = time()
            adj_mat = sp.dok_matrix((self.n_users + self.m_items, self.n_users + self.m_items), dtype=np.float32)
            adj_mat = adj_mat.tolil()
            R = self.UserItemNet.tolil()
            adj_mat[:self.n_users, self.n_users:] = R
            adj_mat[self.n_users:, :self.n_users] = R.T
            adj_mat = adj_mat.todok()
            # adj_mat = adj_mat + sp.eye(adj_mat.shape[0])
            
            rowsum = np.array(adj_mat.sum(axis=1))
            d_inv = np.power(rowsum, -0.5).flatten()
            d_inv[np.isinf(d_inv)] = 0.
            d_mat = sp.diags(d_inv)
            
            norm_adj = d_mat.dot(adj_mat)
            norm_adj = norm_adj.dot(d_mat)
            norm_adj = norm_adj.tocsr()
            end = time()
            print(f"costing {end-s}s, saved norm_mat...")
            sp.save_npz(self.path + '/s_pre_adj_mat.npz', norm_adj)

        if self.split == True:
            self.Graph = self._split_A_hat(norm_adj)
            print("done split matrix")
        else:
            self.Graph = self._convert_sp_mat_to_sp_tensor(norm_adj)
            self.Graph = self.Graph.coalesce().to(world.device)
            print("don't split the matrix")
    return self.Graph

def __build_test(self):
    """
    return:
        dict: {user: [items]}
    """
    test_data = {}
    for i, item in enumerate(self.testItem):
        user = self.testUser[i]
        if test_data.get(user):
            test_data[user].append(item)
        else:
            test_data[user] = [item]
    return test_data

def getUserItemFeedback(self, users, items):
    """
    users:
        shape [-1]
    items:
        shape [-1]
    return:
        feedback [-1]
    """
    # print(self.UserItemNet[users, items])
    return np.array(self.UserItemNet[users, items]).astype('uint8').reshape((-1,))

def getUserPosItems(self, users):
    posItems = []
    for user in users:
        posItems.append(self.UserItemNet[user].nonzero()[1])
    return posItems

In [8]:
# class gowalla(BasicDataset):
#     """ 
#     Dataset type for pytorch include graph information
#     Gowalla Dataset
    
#     """
#   def __init__(self,config = world.config,path="../data/gowalla"):
path="../data/gowalla"
split = config["A_split"]
mode_dict = {'train': 0, "test": 1}
mode = mode_dict['train']
n_user = 0
m_item = 0
train_file = path + '/train.txt'
test_file = path + '/test.txt'
path = path
trainUniqueUsers, trainItem, trainUser = [], [], []
testUniqueUsers, testItem, testUser = [], [], []
trainDataSize = 0
testDataSize = 0

with open(train_file) as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    trainUniqueUsers.append(uid)
                    trainUser.extend([uid] * len(items))
                    trainItem.extend(items)
                    m_item = max(m_item, max(items))
                    n_user = max(n_user, uid)
                    trainDataSize += len(items)
trainUniqueUsers = np.array(trainUniqueUsers)
trainUser = np.array(trainUser)
trainItem = np.array(trainItem)

with open(test_file) as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n').split(' ')
            items = [int(i) for i in l[1:]]
            uid = int(l[0])
            testUniqueUsers.append(uid)
            testUser.extend([uid] * len(items))
            testItem.extend(items)
            m_item = max(m_item, max(items))
            n_user = maxn_user, uid)
            testDataSize += len(items)
m_item += 1
n_user += 1
testUniqueUsers = np.array(testUniqueUsers)
testUser = np.array(testUser)
testItem = np.array(testItem)

Graph = None
print(f"{trainDataSize} interactions for training")
print(f"{testDataSize} interactions for testing")
print(f"{dataset} Sparsity : {(trainDataSize + testDataSize) / n_user / m_item}")

# (users,items), bipartite graph
UserItemNet = csr_matrix((np.ones(len(trainUser)), (trainUser, trainItem)),
                                shape=(n_user, m_item))
users_D = np.array(UserItemNet.sum(axis=1)).squeeze()
users_D[users_D == 0.] = 1
items_D = np.array(UserItemNet.sum(axis=0)).squeeze()
items_D[items_D == 0.] = 1.
# pre-calculate
_allPos = getUserPosItems(list(range(n_user)))
__testDict = __build_test()
print(f"{world.dataset} is ready to go")

loading [../data/gowalla]
